<a href="https://colab.research.google.com/github/sharhp21/ChatBot_SW_Quest/blob/master/stock_price_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

### krx 사이트에 등록되어 있는 상장사 정보(html) 다운로드 받아 dataframe 객체 생성
* krx 상장사 정보 url : http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13

In [0]:
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]

In [0]:
code_df.head(5)

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
1,GS글로벌,1250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서...",1976-06-26,12월,김태형,http://www.gsgcorp.com,서울특별시
2,HSD엔진,82740,일반 목적용 기계 제조업,"대형선박용엔진,내연발전엔진",2011-01-04,12월,고영열,http://www.doosanengine.com,경상남도
3,LG이노텍,11070,전자부품 제조업,기타 전자부품 제조업,2008-07-24,12월,정철동,http://www.lginnotek.co.kr,서울특별시
4,OCI,10060,기초 화학물질 제조업,"타르제품,카본블랙,무수프탈산,농약원제,석탄화학제품,정밀화학제품,플라스틱창호재 제조,판매",1985-07-09,12월,"백우석, 이우현, 김택중(3인, 각자 대표이사)",http://www.oci.co.kr,서울특별시


### Q1. 상장사 정보 중 종목코드를 왼쪽이 '0' 으로 채워진 6자리 값으로 변환하시오

---
hint. map 사용


In [0]:
f1 = lambda x:'%06d'%x
code_df["종목코드"] = code_df["종목코드"].map(f1)
code_df.head(5)

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
1,GS글로벌,001250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서...",1976-06-26,12월,김태형,http://www.gsgcorp.com,서울특별시
2,HSD엔진,082740,일반 목적용 기계 제조업,"대형선박용엔진,내연발전엔진",2011-01-04,12월,고영열,http://www.doosanengine.com,경상남도
3,LG이노텍,011070,전자부품 제조업,기타 전자부품 제조업,2008-07-24,12월,정철동,http://www.lginnotek.co.kr,서울특별시
4,OCI,010060,기초 화학물질 제조업,"타르제품,카본블랙,무수프탈산,농약원제,석탄화학제품,정밀화학제품,플라스틱창호재 제조,판매",1985-07-09,12월,"백우석, 이우현, 김택중(3인, 각자 대표이사)",http://www.oci.co.kr,서울특별시


### Q2. code_df 객체에서 '회사명', '종목코드' 컬럼만 선택하여 code_df 객체에 재 할당 하시오.

In [0]:
code_df = code_df.loc[:,['회사명', '종목코드']]
code_df.head(5)

,회사명,종목코드
0,DSR,155660
1,GS글로벌,001250
2,HSD엔진,082740
3,LG이노텍,011070
4,OCI,010060


### Q3. code_df 객체에서 '회사명' , '종목코드' 컬럼의 이름을 각 각 'name', 'code' 로 변경 하시오

---
hint. rename 사용

In [0]:
code_df.rename(columns={'회사명':'name', '종목코드':'code'}, inplace=True)

### Q4. get_url 함수를 정의하시오
* 입력받은 '회사명'(item_name) 정보와 code_df 객체를 이용하여 '상장 코드' 를 이용하여 naver 주식 시세 정보를 검색하는 url을 작성하시오
* naver 주식 시세 정보 검색 url : https://finance.naver.com/item/sise_day.nhn?code=`상장 코드`
* 작성한 함수를 이용하여 삼성전자 주식 조회를 위한 naver 주식 시세 정보 검색 url 정보를 출력하시오

In [0]:
def get_url(item_name, code_df):
  url = 'https://finance.naver.com/item/sise_day.nhn?code=' + str(code_df)
  return url

In [0]:
# 삼성전자 주식 조회 를 위한 naver 주식 시세정보 검색 url 정보 출력
item_name = '삼성전자'
url = get_url('삼성전자', code_df.loc[code_df['name'] == '삼성전자', 'code'].values[0])

### url을 이용한 주식시세 정보 호출 및 html 파싱을 위한 bs4 라이브러리 임포트

In [0]:
import requests
from bs4 import BeautifulSoup

### Q5. Q4에서 생성한 naver 주식 시세정보 조회 url을 이용하여 200일 간의 삼성전자의 주식 시세 정보를 리스트 객체에 저장하시오
* list 의 shape = (200(일수), 7(날짜, 종가, 차액, 시가, 상한, 하한, 거래량))

In [0]:
df = pd.DataFrame()

priceList = []

for page in range(1, 21):
  # 20*10일 간의 주식 정보를 조회 할 수 있는 url 작성
  url = get_url('삼성전자', code_df.loc[code_df['name'] == '삼성전자', 'code'].values[0]) + '&page=' + str(page)
  
  # url 조회를 통해 얻은 html 정보를 soup 객체를 이용하여 파싱
  page = requests.get(url)
  soup = BeautifulSoup(page.content,'lxml')

  # 날짜, 종가, 차액, 시가, 상한, 하한, 거래량 정보가 담겨 있는 객체 조회
  trList = soup.find_all('tr', onmouseover='mouseOver(this)')

  for tr in trList:
    # 조회된 객체 정보들을 loop 문을 이용해 필요 정보를 선택하여 priceList 객체에 추가
    priceList.append([td.span.string.strip() for td in tr.find_all(name='td')])
print(priceList)

[['2020.01.14', '60,000', '0', '60,400', '61,000', '59,900', '11,973,158'], ['2020.01.13', '60,000', '500', '59,600', '60,000', '59,100', '11,359,139'], ['2020.01.10', '59,500', '900', '58,800', '59,700', '58,300', '16,000,170'], ['2020.01.09', '58,600', '1,800', '58,400', '58,600', '57,400', '24,102,579'], ['2020.01.08', '56,800', '1,000', '56,200', '57,400', '55,900', '23,501,171'], ['2020.01.07', '55,800', '300', '55,700', '56,400', '55,600', '10,009,778'], ['2020.01.06', '55,500', '0', '54,900', '55,600', '54,600', '10,278,951'], ['2020.01.03', '55,500', '300', '56,000', '56,600', '54,900', '15,422,255'], ['2020.01.02', '55,200', '600', '55,500', '56,000', '55,000', '12,993,228'], ['2019.12.30', '55,800', '700', '56,200', '56,600', '55,700', '8,356,767'], ['2019.12.27', '56,500', '1,100', '55,700', '56,900', '55,500', '12,313,056'], ['2019.12.26', '55,400', '400', '54,700', '55,400', '54,400', '9,645,034'], ['2019.12.24', '55,000', '500', '55,600', '55,700', '54,800', '11,868,463']

### Q6. priceList 정보를 이용하여 df 이름을 dataframe 객체 생성
* 컬럼의 이름은 'date', 'close', 'diff', 'open', 'high', 'low', 'volume' 롤 지정

In [0]:
df = pd.DataFrame(priceList, columns=['date', 'close', 'diff', 'open', 'high', 'low', 'volume'])
df.head(5)

,date,close,diff,open,high,low,volume
0,2020.01.14,"60,000",0,"60,400","61,000","59,900","11,973,158"
1,2020.01.13,"60,000",500,"59,600","60,000","59,100","11,359,139"
2,2020.01.10,"59,500",900,"58,800","59,700","58,300","16,000,170"
3,2020.01.09,"58,600","1,800","58,400","58,600","57,400","24,102,579"
4,2020.01.08,"56,800","1,000","56,200","57,400","55,900","23,501,171"


### Q7. df 객체에서 Nan 제거

In [0]:
df.dropna(inplace=True)
df.head()

,date,close,diff,open,high,low,volume
0,2020.01.14,"60,000",0,"60,400","61,000","59,900","11,973,158"
1,2020.01.13,"60,000",500,"59,600","60,000","59,100","11,359,139"
2,2020.01.10,"59,500",900,"58,800","59,700","58,300","16,000,170"
3,2020.01.09,"58,600","1,800","58,400","58,600","57,400","24,102,579"
4,2020.01.08,"56,800","1,000","56,200","57,400","55,900","23,501,171"


### df 객체에 각 원소들의 값이 object로 되어 있기 때문에 분석을 위해 float type으로 

In [0]:
# df 객체 내 컬럼들의 데이터 타입 조회(object))
df.dtypes

date      object
close     object
diff      object
open      object
high      object
low       object
volume    object
dtype: object

In [0]:
# 'close', 'diff', 'open', 'high', 'low', 'volume' 의 값들이
# 천 단위 표현을 위해 ','(5,000) 사용 하고 있음
# 해당 포멧을 숫자로 변환하기 위헤 locale 라이브러리 사용
import locale
from locale import atof

locale.setlocale(locale.LC_NUMERIC, '')

# 날짜의 경우 datetime 타입으로 변환
df['date'] = pd.to_datetime(df['date'])

df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close', 'diff', 'open', 'high', 'low', 'volume']].applymap(atof)
df = df.sort_values(by=['date'], ascending=True)
df.head(10)

,date,close,diff,open,high,low,volume
199,2019-03-26,45250.0,250.0,45500.0,45700.0,44900.0,9729811.0
198,2019-03-27,45350.0,100.0,44750.0,45600.0,44250.0,9568081.0
197,2019-03-28,44850.0,500.0,44950.0,45200.0,44300.0,6821306.0
196,2019-03-29,44650.0,200.0,44500.0,44900.0,44200.0,11491713.0
195,2019-04-01,45050.0,400.0,45200.0,45450.0,44850.0,7362129.0
194,2019-04-02,45750.0,700.0,45550.0,46100.0,45350.0,9480688.0
193,2019-04-03,46600.0,850.0,46750.0,46750.0,45800.0,12436570.0
192,2019-04-04,46950.0,350.0,46150.0,47100.0,46150.0,12650168.0
191,2019-04-05,46850.0,100.0,46950.0,47550.0,46600.0,8546339.0
190,2019-04-08,46650.0,200.0,47250.0,47250.0,46150.0,8507909.0


In [0]:
# df 객체 들의 변환된 데이터 타입 정보 조회(datetime, float64)
df.dtypes

date      datetime64[ns]
close            float64
diff             float64
open             float64
high             float64
low              float64
volume           float64
dtype: object

### 삼성 전자 종가 기준 일별 주가 시계열 차트 조회

In [0]:
# 시계열 차트 생성을 위한 라이브러리 임포트
import plotly.graph_objects as go

In [0]:
# 차트 생성을 위한 데이터 생성
trace = go.Scatter(x=df['date'],
                   y=df['close'],
                   name=item_name)

data = [trace]

In [0]:
# 시계열 차트의 레이아웃 정보 생성
layout = dict(
    title='{}의 종가(close) Time Series'.format(item_name),
    # x축 데이터 타입 지정(date)
    xaxis = dict(
        rangeslider=dict(),
        type='date'
        ),
)

In [0]:
# 차트 객체 생성 및 조회
fig = go.Figure(data=data, layout=layout)
fig.show()

** 프로파일링하여 여러 가지 의미있게 분석된 데이터를 볼 수 있음
---
import pandas_profiling
---
profile = pandas_profiling.ProfileReport(df)
---
profile

In [0]:
import pandas_profiling
profile = pandas_profiling.ProfileReport(df)
profile

/usr/local/lib/python3.6/dist-packages/pandas/plotting/_matplotlib/converter.py:103: FutureWarning:

Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning:

The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.



Number of variables,8
Number of observations,200
Total Missing (%),0.0%
Total size in memory,12.6 KiB
Average record size in memory,64.6 B
Numeric,4
Categorical,0
Boolean,0
Date,1
Text (Unique),0
Rejected,3
